In [1]:
import tkinter as tk
from tkinter import filedialog
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
import numpy as np

# 加载BERT分词器
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 加载BERT模型，并设置output_attentions=True
model = TFBertModel.from_pretrained('bert-base-uncased', output_attentions=True)

def extract_keywords():
    # 打开文件对话框，让用户选择要上传的文件
    file_path = filedialog.askopenfilename(filetypes=[("Text files", "*.txt")])
    
    # 读取用户选择的文件内容
    with open(file_path, 'r') as file:
        text = file.read()
    
    # 将文本数据转换为适合BERT模型输入的格式
    inputs = tokenizer(text, return_tensors='tf', padding=True, truncation=True)

    # 输入文本数据到BERT模型中，获取模型的输出和注意力权重
    outputs = model(inputs)
    attentions = outputs.attentions

    if attentions is not None:
        # 提取关键词
        attention_weights = np.mean(attentions[-1], axis=1)  # 取最后一层的平均注意力权重作为每个词的权重
        max_attention = np.max(attention_weights, axis=-1)  # 获取每个词的最大注意力权重

        # 计算阈值
        percentile = 10
        threshold = np.percentile(max_attention, percentile)

        # 找出超过阈值的词的索引
        keyword_indices = np.where(max_attention > threshold)[0]

        # 将索引列表转换为Python列表
        keyword_indices = keyword_indices.tolist()

        # 根据索引获取关键词
        keywords = [tokenizer.convert_ids_to_tokens(int(inputs['input_ids'][0][idx])) for idx in keyword_indices]

        # 将提取的关键词结果显示在文本框中
        result_text.delete('1.0', tk.END)
        for keyword in keywords:
            result_text.insert(tk.END, f"{keyword}\n")
    else:
        result_text.delete('1.0', tk.END)
        result_text.insert(tk.END, "Attention weights not available for this input.")




# 创建主窗口
root = tk.Tk()
root.title("关键词提取工具")

# 创建上传按钮
upload_button = tk.Button(root, text="上传文件", command=extract_keywords)
upload_button.pack(pady=10)

# 创建显示结果的文本框
result_text = tk.Text(root, height=20, width=50)
result_text.pack()

# 运行主循环
root.mainloop()


ImportError: Traceback (most recent call last):
  File "D:\anaconda\envs\TensorFlow\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: 动态链接库(DLL)初始化例程失败。


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.